<a href="https://colab.research.google.com/github/jfmartineb/EjPrototype/blob/main/Machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Machine learning excercise, Pruebas saber pro
Daniel Gonzalez Bernal - Juan Felipe Martínez Bedoya

This machine learning exercise will be made using decition trees, they are a type of supervised learning algorithm were a training data is split according to a parameter. In this case it is going to be used a CART tree that base from the gini percentage and the information gain of each question. The code to create the tree is the following:

## Settings

We import the libraries that we are going to use along the project

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
import pickle

Line of code used to import files from the computer

In [ ]:
uploades = files.upload()

Saving arbol0.pickle to arbol0.pickle
Saving arbol1.pickle to arbol1.pickle
Saving arbol2.pickle to arbol2.pickle
Saving arbol3.pickle to arbol3.pickle
Saving arbol4.pickle to arbol4.pickle
Saving TEST 0.csv to TEST 0.csv
Saving TEST 1.csv to TEST 1.csv
Saving TEST lite.csv to TEST lite.csv


## Auxiliary methods

- "limpiar" is usefull for tacking the dataset info and eliminates the columns that are not needed
- "valoresunicos" is used to determine if a value given is the only one presented in the rows
- "esnumero" is used to to determine if a value is a number, either integer or float
- "isNaN" is used to determine if a value is equal to pandas NaN
- "armar_arbol" builds the decision tree
- "gini" is used to calculate the Gini value of a row
- "info_gain" is used to calculate the information gain od a given question
- "conteolabels" is used to calculare how many different labels are on a given set of data.
- "dividir" is used to divide information given a question, it classifies it either its true or false
- "mejordivision" gives the best question to make in the decision
- "armar_arbol" is used to build the decision tree
- "imprimir_arbol" is used to print a decition tree given
- "clasificar" is used to take a set of data and determine its label
- "imprimir_hoja" is used to print the leafs of the decition tree
- "probar_certeza" is used to prove how many of the results that the decition tree predicted are correct. The result is given in a percentual value.
- "probar_certeza_bosque" is used to prove how many of the results that the decition forest predicted are correct. The result is given in a percentual value.

In [ ]:
def limpiar(file):
    file.fillna(0)
    matriz = np.matrix(file.values)
    for i in range(len(matriz)):
        s = matriz[i, 18]
        s = s[-4:]
        p = matriz[i, 19]
        p = p // 10
        matriz[i, 18] = int(p) - int(s)
        if matriz[i, 1] == None:
          matriz[i,1] = "SI"
    matriz1 = np.delete(matriz, [0, 11, 12, 22, 24, 48, 50, 57, 58, 62, 64, 73, 74, 76], 1)
    return matriz1.tolist()

def headers(x):
    cabeza = x.columns.values.tolist()
    np.array(cabeza)
    cabeza2 = np.delete(cabeza, [0, 11, 12, 22, 24, 48, 50, 57, 58, 62, 64, 73, 74, 76], 0)
    return cabeza2

def valoresunicos(filas, columna):
    return set([x[columna] for x in filas])

def esnumero(valor):
    return isinstance(valor, int) or isinstance(valor, float)

def isNaN(valor):
  return valor != valor

def gini(fila):
    conteo = conteolabels(fila)
    impureza = 1
    for x in conteo:
        probabilidadexito = conteo[x] / float(len(fila))
        impureza = impureza - probabilidadexito ** 2
    return impureza

def info_gain(izq, der, incertidumbre):
    p = float(len(izq)) / (len(izq) + len(der))
    return incertidumbre - p * gini(izq) - (1 - p) * gini(der)

def conteolabels(filas):
    conteo = {}
    for x in filas:
        label = x[-1]
        if label not in conteo:
            conteo[label] = 0
        conteo[label] += 1
    return conteo

def dividir(filas, pregunta):
    verdadero, falso = [], []
    for x in filas:
        if pregunta.comparar(x):
            verdadero.append(x)
        else:
            falso.append(x)
    return verdadero, falso

def mejordivision(fila):
    best_gain = 0
    best_question = None
    incertidumbreactual = gini(fila)
    n_columnas = len(fila[0]) - 1
    for col in range(n_columnas):
        values = set([x[col] for x in fila])
        for val in values:
            pregunta = Pregunta(col, val)
            verdadero, falso = dividir(fila, pregunta)
            if len(verdadero) == 0 or len(falso) == 0:
                continue
            gain = info_gain(verdadero, falso, incertidumbreactual)
            if gain >= best_gain:
              best_gain = gain
              best_question = pregunta
    return best_gain, best_question

def armar_arbol(filas):
  ganancia, pregunta = mejordivision(filas)

  if ganancia == 0:
    return Hoja(filas)

  filasV, filasF = dividir(filas, pregunta)

  ramaV = armar_arbol(filasV)
  ramaF = armar_arbol(filasF)

  return Nodo_decision(pregunta, ramaV, ramaF)

def imprimir_arbol(nodo, espacio=""):
  if isinstance(nodo, Hoja):
      print(espacio + str(nodo.prediccion))
      return
  print(espacio + str(nodo.pregunta))
  print(espacio + '--> True:')
  imprimir_arbol(nodo.ramaV, espacio + "  ")
  print(espacio + '--> False:')
  imprimir_arbol(nodo.ramaF, espacio + "  ")

def clasificar(fila, nodo):
  if isinstance(nodo, Hoja):
    return nodo.prediccion

  if nodo.pregunta.comparar(fila):
    return clasificar(fila, nodo.ramaV)
  else:
    return clasificar(fila, nodo.ramaF)

def imprimir_hoja(hoja):
  total = sum(hoja.values()) * 1.0
  probs = {}

  for i in hoja.keys():
    probs[i] = str(int(hoja[i] / total * 100)) + "%"

  return probs

def probar_certeza(arbol, filas):
  correctos = 0
  for i in range(len(filas)):
    for k in clasificar(filas[i], arbol):
      if k == filas[i][-1]:
        correctos += 1

  print("La certeza del árbol de decisión es de: %.3f" %((float(correctos)/len(filas))*100),"%")

def probar_certeza_bosque(bosque, filas):
  correctos = 0
  for i in range(len(filas)):
    t = bosque.prediccion(filas[i])
    total = sum(t.values()) * 1.0
    for k in t:
      if k == filas[i][-1]:
        if int(t[k] / total * 100) > 60:
          correctos += 1

  print("La certeza del bosque de decisión es de: %.3f" %((float(correctos)/len(filas))*100),"%")

# Classes

## Question class

In [ ]:
class Pregunta:
    def __init__(self, columna, valor):
        self.columna = columna
        self.valor = valor
    
    def __repr__(self):
       condicion = "=="
       if esnumero(self.valor):
         condicion = ">="
       return "Es la pregunta %s %s %s?" % (header[self.columna], condicion, str(self.valor))

    def comparar(self, ejemplo):
        val = ejemplo[self.columna]
        if isNaN(val):
          return isNaN(self.valor)
        elif esnumero(val):
          return float(val) >= float(self.valor)
        else:
          return val == self.valor

## Leaf class

In [ ]:
class Hoja:
  def __init__(self, filas):
    self.prediccion = conteolabels(filas)

  def __str__(self):
    return str(self.prediccion)


## Decition node

In [ ]:
class Nodo_decision:
  def __init__(self, pregunta, ramaV, ramaF):
    self.pregunta = pregunta
    self.ramaV = ramaV
    self.ramaF = ramaF

## Decition forest

After doing individual trees for the same problem, we make a class that takes all the trees, evaluates all of them and takes the most common decition and takes it as the the new prediction

In [ ]:
class Bosque:
  def __init__(self):
    self.lista_arboles = []

  def agregar_arbol(self, arbol):
    self.lista_arboles.append(arbol)

  def prediccion(self, fila):
    pred = clasificar(fila, self.lista_arboles[0])
    for i in range(1, len(self.lista_arboles)):
      temp = clasificar(fila, self.lista_arboles[i])
      for j in temp:
        if j in pred:
          pred[j] += temp[j]
        else:
          pred[j] = 0
    return pred

# Code used to save and load the trees

This code can be used to save and load the trees, is used to avoid the long compiling times each time we use the program

This code is used to save the decition tree

In [ ]:
pickle.dump(arbol, file = open("arbol.pickle", "wb"))

This code is used to load all the trees for the forest

In [ ]:
arbol0 = pickle.load(open("arbol0.pickle", "rb"))
arbol1 = pickle.load(open("arbol1.pickle", "rb"))
arbol2 = pickle.load(open("arbol2.pickle", "rb"))
arbol3 = pickle.load(open("arbol3.pickle", "rb"))
arbol4 = pickle.load(open("arbol4.pickle", "rb"))

# Code application using a single tree

In this part we will run the code, so it can be tested

First, we turn the uploaded file into pandas data structure

In [ ]:
originalfile = pd.read_csv('TEST lite.csv', sep=';')

Then we create the headers, the code will use them later. CAUTION: Create the headers is a necessary step even if you load the trees from the files. If the tree were loaded from the file it's recomend to create the headers using the TEST lite file to avoid long loading times.

In [ ]:
header = headers(originalfile)

Then we clean the code and create the decition tree from the data. Avoid this step if you loaded the tree from file.

In [ ]:
arbol = armar_arbol(limpiar(originalfile))


We can use this code to print the tree

In [ ]:
imprimir_arbol(arbol1)

Se han truncado las últimas 5000 líneas del flujo de salida.
                          --> False:
                            Es la pregunta cole_cod_dane_sede >= 108001000077.0?
                            --> True:
                              {0: 88}
                            --> False:
                              Es la pregunta cole_cod_mcpio_ubicacion >= 5756.0?
                              --> True:
                                {1: 1}
                              --> False:
                                Es la pregunta punt_biologia >= 52.0?
                                --> True:
                                  Es la pregunta puntaje_prof >= 38.0?
                                  --> True:
                                    {0: 1}
                                  --> False:
                                    {1: 1}
                                --> False:
                                  {0: 12}
                    --> False:
                      {1: 1}
  

Then we create a test data to test the decition tree created, we take the actual value of the original dataset, and compare it to the one that our tree predicts. Also we can give a certainty percentage of the tree.

In [ ]:
prueba = pd.read_csv('TEST 0.csv', sep=';')

In [ ]:
testfile = limpiar(prueba)

In [ ]:
for row in testfile:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], imprimir_hoja(clasificar(row, arbol1))))

Se han truncado las últimas 5000 líneas del flujo de salida.
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted:

In [ ]:
probar_certeza(arbol1, testfile)

La certeza del árbol de decisión es de: 93.520 %


# Code application for decition forest

In this part we will run the code, so it can be tested

First, we pass all the training data to a pandas format so it can be processed (This step is already done since we loaded the trees)

In [ ]:
file0 = pd.read_csv('TRAIN 0.csv', sep=';')
file1 = pd.read_csv('TRAIN 1.csv', sep=';')
file2 = pd.read_csv('TRAIN 2.csv', sep=';')
file3 = pd.read_csv('TRAIN 3.csv', sep=';')
file4 = pd.read_csv('TRAIN 4.csv', sep=';')
file5 = pd.read_csv('TEST 1.csv', sep=';')

Then we create the headers, the code will use them later

In [ ]:
header = headers(prueba)

Then we clean and create the decition tree from the downloaded data (This step is already done since we loaded the trees)

In [ ]:
arbol0 = armar_arbol(limpiar(file0))
arbol1 = armar_arbol(limpiar(file1))
arbol2 = armar_arbol(limpiar(file2))
arbol3 = armar_arbol(limpiar(file3))
arbol4 = armar_arbol(limpiar(file4))

We can check the certainty of each tree individualy

In [ ]:
probar_certeza(arbol0, limpiar(prueba))
probar_certeza(arbol1, limpiar(prueba))
probar_certeza(arbol2, limpiar(prueba))
probar_certeza(arbol3, limpiar(prueba))
probar_certeza(arbol4, limpiar(prueba))


La certeza del árbol de decisión es de: 71.920 %
La certeza del árbol de decisión es de: 93.520 %
La certeza del árbol de decisión es de: 93.440 %
La certeza del árbol de decisión es de: 93.680 %
La certeza del árbol de decisión es de: 93.440 %


After the trees are created, we create the decition forest and add the trees

In [ ]:
bosque = Bosque()
#bosque.agregar_arbol(arbol0)
bosque.agregar_arbol(arbol1)
bosque.agregar_arbol(arbol2)
bosque.agregar_arbol(arbol3)
bosque.agregar_arbol(arbol4)

We test the certainty of the decition forest as we did we the decition tree

In [ ]:
for row in limpiar(prueba):
    print ("Actual: %s. Predicted: %s" %
           (row[-1], imprimir_hoja(bosque.prediccion(row))))

Se han truncado las últimas 5000 líneas del flujo de salida.
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {0: '3%', 1: '96%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 0. Predicted: {0: '100%', 1: '0%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 0. Predicted: {0: '100%'}
Actual: 1. Predicted: {1: '100%'}
Actual: 1. Predicted: {1: '100%', 0: '0%'}
Actual: 1. Predicted: {1: '10

Finally we calculated the percentual certainty that the decition forest have

In [ ]:
probar_certeza_bosque(bosque, limpiar(prueba))

La certeza del bosque de decisión es de: 92.160 %


# Conclutions

- Its better to load the decition trees from files taken from previous runs, so the code does not take long creating the decition object.
- A crucial part for a machine learning code is the pre processing of the data, given this part the decition tree cration will vary, and the better the pre processing, the better the tree.
- To train the code is better to use bigger data sets, to assure a better accuracy of the decition tree.
- It was found that its more efficient to create the decition forest from individual trees that have a high certainty percentage, and avoid using trees that have it low, so they do not affect the global accuracy of the forest.

# Bibliography

Let’s Write a Decision Tree Classifier from Scratch - Machine Learning Recipes #8 - From Google Developers channel -
 https://youtu.be/LDRbO9a6XPU

https://github.com/random-forests/tutorials - Random forest tutorial by Josh Gordon

https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/ by Jason Brownlee PhD

https://stackoverflow.com/questions/4529815/saving-an-object-data-persistence